In [52]:
#EXAMPLE CODE FROM HYPEROPT DOCS
# define an objective function
def objective(args):
    case, val = args
    if case == 'case 1':
        return val
    else:
        return val ** 2

# define a search space
space = hp.choice('a',
    [
        ('case 1', 1 + hp.lognormal('c1', 0, 1)),
        ('case 2', hp.uniform('c2', -10, 10))
    ])

# minimize the objective over the space
best = fmin(objective, space, algo=tpe.suggest, max_evals=100)

print(best)
# -> {'a': 1, 'c2': 0.01420615366247227}
print(space_eval(space, best))

100%|██████████| 100/100 [00:00<00:00, 408.92trial/s, best loss: 6.357251849118012e-07]
{'a': 1, 'c2': -0.0007973237641709929}
('case 2', -0.0007973237641709929)


In [133]:
import pandas as pd
import numpy as np
from hyperopt import hp, fmin, tpe, space_eval
from scikits.odes.ode import ode
import random

In [122]:
#Import local functions
sys.path.append('../functions')
from functions import nonlinearity, repression, activation

In [86]:
#TOY OBJECTIVE OPTIMIZATION LOOP - getting weird results
#Define search space based on A matrix
E = 2
T = [1]
space = {e: hp.choice(str(e), ['activation', 'repression', 'none']) for e in range(E)}

#Define objective function
def objective(args):
    case, val = args
    if case == 'activation':
        return val
    else:
        return val ** 2

# minimize the objective over the space
best = fmin(objective, space, algo=tpe.suggest, max_evals=1)
print(best)
print(space_eval(space, best))

100%|██████████| 1/1 [00:00<00:00, 606.46trial/s, best loss: 1.0]
{'0': 1, '1': 0}
{0: 'repression', 1: 'activation'}


In [126]:
#Define search space based on A matrix - for now this code assumes a single TF
choices = ['activation', 'repression', 'none']
search_space = {e: hp.choice(str(e), choices) for e in range(E)}

algo = tpe.suggest

def train(params):
    #Instantiate new architecture matrix
    A = np.zeros([len(T), E, 3]) 

    #One-hot encode choices from hyperopt
    one_hot = pd.get_dummies(pd.Series(choices))

    #Fill in architecture matrix with choices
    for i in range(len(params)):
        A[0][i] = one_hot[params[i]].to_list()

    loss = solve_architecture(A)
    return loss 

def solve_architecture(A):
    return 1 #Dummy function for now
    
best_hyperparams = fmin(fn=train, space=search_space, algo=algo, max_evals=1)

100%|██████████| 1/1 [00:00<00:00, 162.94trial/s, best loss: 1.0]


In [124]:
#Default information - user can change but mostly won't
#Constants (set to default)
kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467

#Integration conditions
y = np.array([2290.,0.,1.,0.])
t = np.linspace(0, 5e4, 100) 

#User specified information
X = 2 #Number of chemical species in pathway
E = 2 #Number of enzymes in pathway

#S = np.zeros([X, E])
S = np.array([[ -1, 0], [1, -1]]) #Stoichiometric matrix defining chemical reactions

I = np.zeros(X) #Input matrix defining inputs to pathway
I[0] = Vin #Inputs to pathway

T = [1] #Xs that can act as TFs

#Genetic parameter values for circuit
W = np.zeros([len(T), E, 3]) #K, theta, n are 3 dimensions

#Manually set initial conditions
for t in range(len(T)):
    for e in range(E):
        W[t][e] = [2., 0.1,1E-4]

def generated_equation(t, y, ydot):
    '''Generate genetic equations from architecture matrix'''
    ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
    ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
    for e in range(E):
        ydot[e+X] = -lam*y[e+X] + np.sum([np.sum(A[t][e]*[activation(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), repression(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), W[t][e][2]]) for t in range(len(T))])
    ydot[E+X] = (I[0] - y[X]*nonlinearity(y[X-1], kc, km))**2 #J1
    ydot[E+X+1] = np.sum([np.sum([np.sum(A[t][e]*[activation(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), repression(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), 0]) for t in range(len(T))]) for e in range(E)]) #J2

In [136]:
def solve_point(x):
    n1, n2, theta1, theta2, k1, k2 = x
    
    alpha1 = 1E-5
    alpha2 = 1E-2

    int_time = 5E4
    t = np.linspace(0, int_time, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.]) 

    def dual_control_sundials(t, y, ydot):
        kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
        ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = repression(y[1], k1, theta1, n1) + activation(y[1], k2, theta2, n2)

    def loss_biological(j1, j2, alpha1, alpha2):
        """Computes scalarized loss including genetic constraint and product production"""
        loss = alpha1*j1 + alpha2*j2
        return j1, j2, loss
        
    solution = ode('cvode', dual_control_sundials, old_api=False).solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2, alpha1=alpha1, alpha2=alpha2)
    return loss

def solve_patch_6d(init_conds, patch_size, step_sizes):
    n1, n2, theta1, theta2, k1, k2 = init_conds
    center_loss = solve_point(init_conds)
    #Sample randomly from hypersphere
    normal_deviates = np.random.normal(size=(6, patch_size))
    radius = np.sqrt((normal_deviates**2).sum(axis=0))
    points = normal_deviates/radius

    scaled_points = []
    for i in range(patch_size):
        scaled_points.append(list(step_sizes*points[:, i]  + init_conds))
    scaled_points = np.array(scaled_points)
    
    min_loss = center_loss
    new_k1 = k1; new_k2 = k2; new_theta1 = theta1; new_theta2 = theta2; new_n1 = n1; new_n2 = n2
    for n1, n2, theta1, theta2, k1, k2 in zip(scaled_points[:, 0], scaled_points[:, 1], scaled_points[:, 2], scaled_points[:, 3], scaled_points[:, 4], scaled_points[:, 5]):
        loss = solve_point([n1, n2, theta1, theta2, k1, k2])
        if bounds_check_6d(n1, n2, theta1, theta2, k1, k2):
            if loss < min_loss:
                new_k1 = k1
                new_k2 = k2
                new_theta1 = theta1
                new_theta2 = theta2
                new_n1 = n1
                new_n2 = n2
                min_loss = loss
    return new_n1, new_n2, new_theta1, new_theta2, new_k1, new_k2, min_loss

def bounds_check_6d(n1, n2, theta1, theta2, k1, k2):
    if k1 <= 1E-3 and k2 <= 1E-3 and k1 >= 1E-7 and k2 >= 1E-7:
        if n1 <= 4 and n2 <= 4 and n1 >= 1 and n2 >= 1:
            if theta1 >= 0.001 and theta2 >= 0.001 and theta1 <= 10 and theta2 <= 10:
                return True
    else: return False

def gen_ics(N):
    #Set random seed
    np.random.seed(2022)
    ics = []
    for i in range(N):
        k1 = random.uniform(1E-7,1E-3)
        k2 = random.uniform(1E-7,1E-3)
        n1 = random.uniform(1,4)
        n2 = random.uniform(1,4)
        theta1 = random.uniform(0.001,10)
        theta2 = random.uniform(0.001,10)

        init_conds = np.array([n1,n2,theta1, theta2, k1, k2])
        ics.append(init_conds)
    return ics

In [135]:
def run_descent(ics, num_epochs = 100, tolerance=0.0004, patch_size=100, step_sizes= np.array([0.1, 0.1, 0.01, 0.01, 0.0001, 0.0001])):
    list_of_results = []
    for i in range(len(ics)):
        print('Descent', i+1, 'of', len(ics))
        init_conds = ics[i]
        solve_patch_6d(init_conds, patch_size, step_sizes)
        next_n1, next_n2, next_theta1, next_theta2, next_k1, next_k2, loss = solve_patch_6d(init_conds, patch_size, step_sizes)

        losses = [loss]; k1_trace = [next_k1]; k2_trace = [next_k2]; n1_trace = [next_n1]; n2_trace = [next_n2]; theta1_trace = [next_theta1]; theta2_trace = [next_theta2]

        for i in range(num_epochs):
            new_n1, new_n2, new_theta1, new_theta2, new_k1, new_k2, loss = solve_patch_6d([next_n1, next_n2, next_theta1, next_theta2, next_k1, next_k2], patch_size, step_sizes)
            next_k1 = new_k1
            next_k2 = new_k2
            next_theta1 = new_theta1
            next_theta2 = new_theta2
            next_n1 = new_n1
            next_n2 = new_n2

            print('Epoch: ', i, 'Loss: ', loss)
            losses.append(loss)
            k1_trace.append(new_k1)
            k2_trace.append(new_k2)
            n1_trace.append(new_n1)
            n2_trace.append(new_n2)
            theta1_trace.append(new_theta1)
            theta2_trace.append(new_theta2)

            if losses[-2] - losses[-1] < tolerance:
                print('Terminating early at step', i, 'of ', num_epochs)
                break
        results = pd.DataFrame({'loss':losses,'k1_trace':k1_trace,'k2_trace':k2_trace,'n1_trace':n1_trace,'n2_trace':n2_trace,'theta1_trace':theta1_trace, 'theta2_trace':theta2_trace})
        list_of_results.append(results)


In [139]:
def solve_architecture(A):
    #Randomize initial conditions 
    ics = gen_ics(1)
    run_descent(ics)

Descent 1 of 1
Epoch:  0 Loss:  0.6261115866013945
Epoch:  1 Loss:  0.2470640412251493
Epoch:  2 Loss:  0.06163620585678996
Epoch:  3 Loss:  0.050028061249718266
Epoch:  4 Loss:  0.04942104163839573
Epoch:  5 Loss:  0.048670563016563216
Epoch:  6 Loss:  0.04830385915212845
Terminating early at step 6 of  100


In [ ]:
def solve_point(x):
    n1, n2, theta1, theta2, k1, k2 = x
    
    alpha1 = 1E-5
    alpha2 = 1E-2

    int_time = 5E4
    t = np.linspace(0, int_time, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.]) 

    def dual_control_sundials(t, y, ydot):
        kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
        ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = repression(y[1], k1, theta1, n1) + activation(y[1], k2, theta2, n2)

    def loss_biological(j1, j2, alpha1, alpha2):
        """Computes scalarized loss including genetic constraint and product production"""
        loss = alpha1*j1 + alpha2*j2
        return j1, j2, loss
        
    solution = ode('cvode', dual_control_sundials, old_api=False).solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2, alpha1=alpha1, alpha2=alpha2)
    return loss

In [141]:
from solve_architecture import solve_architecture

In [142]:
def main():
    #Integration conditions
    y0 = np.array([2290., 0., 0., 0., 0., 0.]) 
    times = np.linspace(0, 5e4, 100) 

    A = np.zeros([1, 2, 3]) #3 methods of action at each locus for each TF
    A[0][0] = [0, 1, 0]
    A[0][1] = [0, 0, 1]

    param_trace, losses = solve_architecture(2, 2, [1], A, times, y0, num_initializations=1, num_epochs = 100, tolerance=0.0004, patch_size=100, step_sizes= np.array([[[0.1, 0.01, 0.001], [0.1, 0.01, 0.0001]]]))
   
main()

Descent 1 of 1
Epoch:  0 Loss:  0.1213380630702483


/Users/charlotte/Library/Mobile Documents/com~apple~CloudDocs/Documents/PhD/Thesis/msc-thesis/model_building/solve_architecture.py:8: RuntimeWarning: invalid value encountered in double_scalars
  return (k*(x/theta)**n)/(1+(x/theta)**n)
/Users/charlotte/Library/Mobile Documents/com~apple~CloudDocs/Documents/PhD/Thesis/msc-thesis/model_building/solve_architecture.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return k/(1+(x/theta)**n)

[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.



Epoch:  1 Loss:  0.04570176647192317



[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13

Epoch:  2 Loss:  0.04463523232392286



[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13

Epoch:  3 Loss:  0.04462075701699269
Terminating early at step 3 of  100



[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.

